In [24]:
import numpy as np

# Returns a copy of data which is normalized around mean of spinal base point
def normalize_spine_base(data):
    b = np.copy(data)
    spine = np.zeros(3)
    for i in range(3):
       spine[i] = np.mean(b[:, i])
    m, n = b.shape
    for i in range(n):
       b[:, i] -= spine[i % 3]
    return b

# Pulls up torso skeleton data from a general skeleton file, and normalizes it using normalize_spine_data()
def load_skeleton_data(filename):
    data = np.loadtxt(filename, dtype='float', delimiter=',', skiprows=1, usecols=(9, 10, 11, 18, 19, 20, 27, 28, 29, 36, 37, 38, 45, 46, 47, 54, 55, 56, 63, 64, 65, 72, 73, 74, 81, 82, 83, 90, 91, 92, 99, 100, 101, 108, 109, 110, 189, 190, 191))
    return normalize_spine_base(data)

In [25]:
import os, random
from pathlib import Path

# Chosse a random skeleton file for a given gesture
def choose_random_skeleton(gesture):
    current_dir = Path('.')
    data_dir = current_dir / 'data' / gesture
    return random.choice([str(f) for f in list(data_dir.glob("*.txt")) if f.is_file()])


In [26]:
# Number of samples to be taken from each gesture class
tot_samples = 3

In [27]:
# Taking tot_samples number of samples from both gesture classes
x_files = [ choose_random_skeleton('head_nod') for i in range(tot_samples) ]
y_files = [ choose_random_skeleton('arms_move_down') for i in range(tot_samples) ]

In [28]:
from dtw import dtw

# Generating a similarity matrix from two list of files with equal length
def gen_sim_matrix(x_files, y_files):
    # Get a combined list of the skeleton files chosen earlier
    combined_files = x_files + y_files
    # Initializing the similarity matrix to be used for final comparison
    s_matrix = np.zeros( len(combined_files) ** 2).reshape(len(combined_files), len(combined_files))
    # Populating the similarity matrix with distance outputs from DTW
    for i, fi in enumerate(combined_files):
        fi_data = load_skeleton_data(fi)
        for j, fj in enumerate(combined_files):
            fj_data = load_skeleton_data(fj)
            dist, cost, acc, path = dtw(fi_data, fj_data, dist=lambda a, b: np.linalg.norm(a - b))
            s_matrix[i, j] = dist
    return s_matrix

# Generates list of tuples of similar files skeletons
def gen_sim_files(x_files, y_files, s_matrix):
    sim_files = []
    # Get a combined list of the skeleton files chosen earlier
    combined_files = x_files + y_files
    s_matrix_c = np.copy(s_matrix)
    for i in range(len(s_matrix_c)):
        # Need to do this so that 0 entries along the diagonal are not chosen as the minimum
        s_matrix_c[i,i] = float("inf")
        min_index = np.argmin(s_matrix_c[i])
        sim_files += [ (combined_files[i], combined_files[min_index])]
    return sim_files

# Writes a playlist based on the list of files it gets
# Make sure that files is a list of valid string paths
def write_playlist(files):
    # Creating a playlist file
    similar_playlist = open('similar.pls', 'w')
    
    # Header of the .pls format
    similar_playlist.write('[playlist]\n\n')
    
    for i in range(len(files)):
        similar_playlist.write('File' + str(i+1) + '=' + files[i] + '\n\n')
    
    # The footer for .pls format
    similar_playlist.write('NumberOfEntries=' + str(len(files)) + '\n')
    similar_playlist.write('Version=2')
    similar_playlist.close()

In [29]:
print("Samples chosen:\n")
for i in x_files + y_files:
    print(i)

print()
print("Similarity matrix is:\n")
s_matrix = gen_sim_matrix(x_files, y_files)
print(s_matrix)
print()

s_files = gen_sim_files(x_files, y_files, s_matrix)
print("Similar files are:\n")
for i in s_files:
    print(i[0])
    print(i[1])
    print()

# Modifying s_files for use into a playlist
# Writing playlist entries for similar files one after the other
# Trimming '_skeleton.txt' from the end and appending '.avi' suitable for the .pls file        
s_v_files = [ tp[i][:-13]+'.avi' for tp in s_files for i in range(len(tp)) ]

# Writing a pls file
write_playlist(s_v_files)

Samples chosen:

data\head_nod\Gesture52_Session 2-Participant 3-Block 6_975365227_Skeleton.txt
data\head_nod\Gesture52_Session 20-Participant 40-Block 2_1315500000_Skeleton.txt
data\head_nod\Gesture52_Session 20-Participant 40-Block 3_860000000_Skeleton.txt
data\arms_move_down\Gesture968_Session 7-Participant 13-Block 3_4242670000_Skeleton.txt
data\arms_move_down\Gesture968_Session 3-Participant 5-Block 3_1142670000_Skeleton.txt
data\arms_move_down\Gesture968_Session 20-Participant 39-Block 5_2822670000_Skeleton.txt

Similarity matrix is:

[[ 0.          0.75691463  0.38842556  0.32720199  0.57865279  0.25958267]
 [ 0.75691463  0.          0.51860902  0.75727304  0.63400575  0.58715963]
 [ 0.38842556  0.51860902  0.          0.62015651  0.39113679  0.40169206]
 [ 0.32720199  0.75727304  0.62015651  0.          0.47352047  0.25952326]
 [ 0.57865279  0.63400575  0.39113679  0.47352047  0.          0.41209067]
 [ 0.25958267  0.58715963  0.40169206  0.25952326  0.41209067  0.        ]]

S